In [1]:
import joblib

# chargement data 
import pandas as pd
original_csv = pd.read_csv('static/films_jp_box.csv')
original_csv.head(2)

,film_id,titre,genre_principale,date_sortie_france,date_sortie_usa,image_url,synopsis,duree,note_moyenne,acteurs,entrees_demarrage_france,entrees_totales_france,budget,recette_usa,recette_reste_du_monde,recette_monde
0,20156,L'Extraordinaire Voyage de Marona,Animation,08/01/2020,00/00/0000,https://www.jpbox-office.com/cinema/images/pos...,"Victime d’un accident, Marona, une petite chie...",1h 32min,Non disponible,Non disponible,12 065,37 632,?,- $,41 568 $,41 568 $
1,19778,Les Filles du Docteur March,Adaptation Livre,01/01/2020,25/12/2019,https://www.jpbox-office.com/cinema/images/pos...,Une nouvelle adaptation du classique de Louisa...,2h 15min,3,Emma Watson (Rôle principal - - Meg March) | T...,242 663,805 211,40 000 000 $,108 101 214 $,92 100 000 $,200 201 214 $


In [2]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "filmprediction.settings")
django.setup()

from films.models import Movie
from asgiref.sync import sync_to_async

@sync_to_async
def get_movie(movie_id) -> Movie :
    return Movie.objects.get(id=movie_id)

selected_movie_id = 7
selected_movie = await get_movie(selected_movie_id)

print(selected_movie)


Gran Turismo


In [3]:
movie_row = original_csv[original_csv['titre']== selected_movie.title].iloc[[0]]

from sklearn.pipeline import Pipeline

dummy_model: Pipeline  = joblib.load("dummy_pipeline.joblib")
prediction = dummy_model.predict(movie_row)


text_line  = f"{selected_movie.title} : {int(prediction)} entrées en première semaine"
print(" __"+len(text_line)*"_"+"__ ")
print("|  "+len(text_line)*" "+"  |")
print("|  " + text_line + "  |")
print("|__"+len(text_line)*"_"+"__|")
print(movie_row['entrees_demarrage_france'])


 _____________________________________________________ 
|                                                     |
|  Gran Turismo : 291516 entrées en première semaine  |
|_____________________________________________________|
6    382 349
Name: entrees_demarrage_france, dtype: object


/home/nicolascassonnet/Documents/WORK/film-prediction/django/model_utils.py:61: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return date_columns.applymap(get_year).to_numpy()
/home/nicolascassonnet/Documents/WORK/film-prediction/django/model_utils.py:64: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return date_columns.applymap(get_minutes).to_numpy()
/home/nicolascassonnet/Documents/WORK/film-prediction/django/model_utils.py:67: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return number_columns.applymap(get_number).to_numpy()
/tmp/ipykernel_26211/337416984.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  text_line  = f"{selected_movie.title} : {int(prediction)} entrées en première semaine"


In [10]:
# mise à jour de la base de données

from films.models import Movie, PredictionHistory
import datetime as dt

pred_int = prediction
today = dt.datetime.now().date()

new_entry = PredictionHistory(
    movie_id = selected_movie.id, 
    first_week_predicted_entries_france = pred_int, 
    metric_score = float(pred_int/4),
    model_version = 0,
    date = today
)

@sync_to_async
def insert_prediction_history(prediction_history : PredictionHistory):
    prediction_history.save()

await insert_prediction_history(new_entry)

@sync_to_async
def get_movie_and_predictions(movie_id) -> Movie:
    return_movie = Movie.objects.prefetch_related('predictions').get(id=movie_id)
    return return_movie

updated_movie = await get_movie_and_predictions(selected_movie.id)
predictions = updated_movie.predictions.all()

print (f"predictions : {len(predictions)}")


predictions : 16


/tmp/ipykernel_26211/2547110217.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  metric_score = float(pred_int/4),
